In [510]:
import numpy as np
import scipy as sc

from src.Tools import portfolio_metrics
from qiskit_src.ansatz import CP_QAOA, qubo_cost
from qiskit_src.tools import get_qubo, min_cost_partition

In [534]:
N=10
k=5
seed=3
alpha=0.001
layers=5
max_iter=100
w_z_phase = False

In [535]:
expected_returns, covariances = portfolio_metrics(n=N, seed=seed)

constrained_result, full_result, lmbda = min_cost_partition(nr_qubits=N,
                                                            k=k,
                                                            mu=expected_returns,
                                                            sigma=covariances,
                                                            alpha=alpha)

max_cost, min_cost, min_state = constrained_result['c_max'], constrained_result['c_min'], constrained_result['s']
min_state_str = ''.join([str(_) for _ in min_state])
Q, offset = get_qubo(mu=expected_returns,
                     sigma=covariances,
                     alpha=alpha,
                     lmbda=lmbda,
                     k=k)

In [540]:


_available_methods_ = ['COBYLA', 'Nelder-Mead']
_method_idx_ = 0

ansatz = CP_QAOA(N_qubits=N,
                 cardinality=k,
                 layers=layers,
                 QUBO_matrix=Q,
                 QUBO_offset=offset,
                 with_z_phase=w_z_phase)

# Initial guess for parameters (gamma, beta) of circuit
theta_min, theta_max = -np.pi, np.pi
N_xx_yy_angles = layers * (N - 1) 
if w_z_phase:
    N_xx_yy_angles += N * layers
#theta_i = np.random.uniform(low=theta_min, high=theta_max, size=N_xx_yy_angles)
theta_i = np.random.normal(loc=0, scale=1, size=N_xx_yy_angles)
# Use the get_cost method of the specific ansatz instance
res = sc.optimize.minimize(fun=ansatz.get_cost, x0=theta_i,
                           method=_available_methods_[_method_idx_],
                           options={'disp': False, 'maxiter': max_iter})

_dict_ = ansatz.get_state_probabilities(angles=res.x, flip_states=False)
res

 message: Maximum number of function evaluations has been exceeded.
 success: False
  status: 2
     fun: -0.10322226877182035
       x: [-1.196e+00 -5.455e-01 ... -7.844e-01  9.424e-01]
    nfev: 100
   maxcv: 0.0

In [541]:
best_state = list(_dict_.keys())[np.argmax(list(_dict_.values()))]
best_state, _dict_[best_state] 

('1100110001', 0.4010485076497)

In [542]:
min_state_str

'1101110000'

In [543]:
_dict_[min_state_str] 

0.001185346874596

In [517]:
opt_cost = qubo_cost(min_state.astype(np.float64),Q)

In [518]:
found_cost = qubo_cost(np.array([float(_) for _ in best_state]).astype(np.float64), Q)

In [519]:
print(f'Deviation from optimal cost: {round(100*((opt_cost/found_cost) - 1),3)} %')

Deviation from optimal cost: 0.49 %
